In [23]:
import pandas as pd
import numpy as np

In [24]:
donations = pd.read_pickle('out/21/donations.pkl')
us_states = pd.read_pickle('out/11/states_gps.pkl')
us_counties = pd.read_pickle('out/11/counties_gps.pkl')
population = pd.read_pickle('out/11/indian_population.pkl')

In [25]:
population.head()

,state,county_norm,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015
0,AL,autauga,101,92,83,74,64,55,46,37,27,18,9,0,0,0,0
1,AL,baldwin,69,72,74,77,79,82,84,87,89,92,94,97,99,102,104
2,AL,barbour,0,0,0,0,2,5,7,10,18,26,34,42,50,58,66
3,AL,bibb,33,30,27,24,21,18,15,12,9,6,3,0,0,0,0
4,AL,blount,211,192,173,154,134,115,96,77,57,38,19,0,0,0,0


In [26]:
yearly_india_pop = population.set_index(['state', 'county_norm']).apply(sum)
yearly_donor_pop = donations.groupby('activity_year').donor_id.nunique()
yearly_mean_donor_fraction = yearly_donor_pop*1.0/yearly_india_pop

yearly_expected_donors = population.set_index(['state', 'county_norm']).mul(yearly_mean_donor_fraction, axis=1).astype('int')
yearly_expected_donors = yearly_expected_donors.reset_index()

In [27]:
yearly_actual_donors = donations\
    .groupby(['state', 'county_norm', 'activity_year'])\
    .donor_id\
    .nunique()\
    .to_frame()\
    .unstack()\
    .fillna(0)

# drop the "donor_id" dummy level
yearly_actual_donors.columns = yearly_actual_donors.columns.droplevel()

In [28]:
# rows don't match
yearly_expected_donors.shape, yearly_actual_donors.shape, population.shape

((3109, 17), (756, 15), (3109, 17))

In [29]:
# To make things easier later, we want to set the values for all the counties in the yearly actual donor dataframe
yearly_actual_donors = yearly_actual_donors.reset_index()\
    .merge(population.reset_index()[['state', 'county_norm']],
           on=['state', 'county_norm'],
           how='right')\
    .sort_values(by='county_norm')\
    .fillna(0)


In [30]:
# rows match after conversion
yearly_expected_donors.shape, yearly_actual_donors.shape, population.shape

((3109, 17), (3109, 17), (3109, 17))

In [31]:
cols = yearly_expected_donors.columns.values[2:]

In [32]:
cols = np.append(cols, ['state', 'county_norm'])

In [33]:
yearly_expected_donors = yearly_expected_donors\
    .sort_values(by=['state', 'county_norm'])\
    .reset_index(drop=True)[cols]

In [34]:
yearly_actual_donors = yearly_actual_donors\
    .sort_values(by=['state', 'county_norm'])\
    .reset_index(drop=True)[cols]

In [35]:
!mkdir -p out/40
yearly_expected_donors.to_pickle('out/40/yearly_expected_donors.pkl')
yearly_actual_donors.to_pickle('out/40/yearly_actual_donors.pkl')

### Testing out queries for the web app

In [36]:
indian_population = population

In [37]:
statesfilter = ['CA', 'WA']
yearly_actual_donors[yearly_actual_donors.state.isin(statesfilter)]
yearly_expected_donors[yearly_expected_donors.state.isin(statesfilter)]
_ = indian_population[indian_population.state.isin(statesfilter)]

In [38]:
yearly_expected_donors.head()

,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,state,county_norm
0,0,3,6,6,6,10,11,12,12,11,11,9,8,11,5,0,districtofcolumbia
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,AL,autauga
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,AL,baldwin
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,AL,barbour
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,AL,bibb


In [39]:
actual =  yearly_actual_donors[yearly_actual_donors.state.isin(statesfilter)]
expected =  yearly_expected_donors[yearly_expected_donors.state.isin(statesfilter)]
population =  indian_population[indian_population.state.isin(statesfilter)]

In [40]:
difference = actual.set_index(['state', 'county_norm']) - expected.set_index(['state', 'county_norm'])

In [41]:
population[population.county_norm=='snohomish']

,state,county_norm,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015
2949,WA,snohomish,1329,1700,2071,2443,2814,3185,3556,3928,4299,4670,5041,5413,5784,6155,6526


In [42]:
population =  indian_population[indian_population.state.isin(statesfilter)]

In [43]:
year = 2014
statesfilter = ['AZ', 'CA', 'CO', 'ID', 'MT', 'NM', 'NV', 'OR', 'UT', 'WA', 'WY']
print statesfilter
population[population.state.isin(statesfilter)][year]
population.query('state in @statesfilter')[['county_norm', year]]

['AZ', 'CA', 'CO', 'ID', 'MT', 'NM', 'NV', 'OR', 'UT', 'WA', 'WY']


,county_norm,2014
157,alameda,65143
158,alpine,0
159,amador,16
160,butte,545
161,calaveras,18
162,colusa,0
163,contracosta,19034
164,delnorte,0
165,eldorado,1251
166,fresno,12994


In [44]:
population.county_norm.nunique()

97